In [3]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
df.dropna(subset=["Question", "Answer", "Category"], inplace=True)
df["Category"] = df["Category"].str.lower().str.strip()

# Function to evaluate the answer
def evaluate_answer(user_answer, correct_answer):
    vectorizer = TfidfVectorizer().fit_transform([user_answer, correct_answer])
    similarity = cosine_similarity(vectorizer[0:1], vectorizer[1:2])[0][0]
    score = round(similarity * 100)
    return score, similarity

# Main function to run the interview mentor
def run_interview_mentor():
    print("Welcome to the AI Interview Mentor!\n")
    user_name = input("Enter your name: ")
    print(f"\nHi {user_name}, let's get you interview-ready!")

    available_domains = df["Category"].unique()
    print("\nAvailable categories:", ", ".join(domain.title() for domain in available_domains))

    selected_domain = input("\nEnter your preferred category: ").lower().strip()
    if selected_domain not in available_domains:
        print("Sorry, we don't have questions for that category.")
        return

    domain_questions = df[df["Category"] == selected_domain]
    selected_qas = domain_questions.sample(n=min(5, len(domain_questions)))

    total_score = 0
    for i, row in enumerate(selected_qas.itertuples(), 1):
        print(f"\nQuestion {i}: {row.Question}")
        user_answer = input("Your answer: ")

        score, similarity = evaluate_answer(user_answer, row.Answer)
        total_score += score

        print("--- Feedback ---")
        print(f"Similarity: {similarity:.2f}, Score: {score}/100")
        if score < 30:
            print("Feedback: Your answer is quite different. Try to align more with key concepts.")
        elif score < 70:
            print("Feedback: Decent answer, but it misses some important details.")
        else:
            print("Feedback: Great answer! You captured the main points well.")

    average_score = total_score / len(selected_qas)
    print(f"\n{user_name}, your average interview score for '{selected_domain.title()}' is: {average_score:.2f}/100")

# Entry point of the program
if __name__ == "__main__":
    run_interview_mentor()


Welcome to the AI Interview Mentor!




Hi varun, let's get you interview-ready!

Available categories: General Programming, General Program, Data Structures, Languages And Frameworks, Database And Sql, Web Development, Software Testing, Version Control, System Design, Security, Devops, Front-End, Back-End, Full-Stack, Algorithms, Machine Learning, Distributed Systems, Networking, Low-Level Systems, Database Systems, Data Engineering, Artificial Intelligence
Sorry, we don't have questions for that category.


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Excel files
students = pd.read_excel("students.xlsx")
opportunities = pd.read_excel("opportunities.xlsx")

# Combine features for matching
students['profile'] = students['skills'] + ' ' + students['interests'] + ' ' + students['location']
opportunities['requirement'] = opportunities['required_skills'] + ' ' + opportunities['field'] + ' ' + opportunities['location']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
student_vecs = vectorizer.fit_transform(students['profile'])
opportunity_vecs = vectorizer.transform(opportunities['requirement'])

# Cosine similarity
similarity = cosine_similarity(student_vecs, opportunity_vecs)

# Recommend top 3 matches for each student
for i, student in students.iterrows():
    top_indices = similarity[i].argsort()[-3:][::-1]
    print(f"\nTop matches for Student {student['student_id']}:")
    for idx in top_indices:
        opp = opportunities.iloc[idx]
        print(f"  - Opportunity {opp['opportunity_id']} (Score: {similarity[i][idx]:.2f})")


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Hidden personality labels linked with careers (for internal model use only)
personality_data = {
    "Strategic and Logical (likes planning and analyzing)": "INTJ",
    "Creative and Outgoing (enjoys new ideas and people)": "ENFP",
    "Hands-on and Practical (good with tools and machines)": "ISTP",
    "Caring and Social (loves helping others)": "ESFJ",
    "Imaginative and Quiet (writes or creates deeply)": "INFP",
    "Curious and Energetic (loves solving problems)": "ENTP",
    "Helpful and Organized (likes structure and stability)": "ISFJ",
    "Efficient and Direct (natural leader)": "ESTJ",
    "Confident and Driven (leads with vision)": "ENTJ",
    "Empathetic and Insightful (understands people deeply)": "INFJ",
    "Fun and Outgoing (loves performance and spotlight)": "ESFP",
    "Gentle and Artistic (prefers calm and expression)": "ISFP",
    "Bold and Active (enjoys action and business)": "ESTP",
    "Detail-oriented and Responsible (follows rules and tasks)": "ISTJ"
}

career_data = {
    "INTJ": "Software Engineer",
    "ENFP": "Marketing Manager",
    "ISTP": "Mechanical Engineer",
    "ESFJ": "Teacher",
    "INFP": "Writer",
    "ENTP": "Entrepreneur",
    "ISFJ": "Nurse",
    "ESTJ": "Project Manager",
    "ENTJ": "Executive",
    "INFJ": "Psychologist",
    "ESFP": "Performer",
    "ISFP": "Artist",
    "ESTP": "Sales Executive",
    "ISTJ": "Accountant"
}

# Create training DataFrame
df = pd.DataFrame({
    "personality_description": list(personality_data.keys()),
    "personality_code": list(personality_data.values()),
    "preferred_career": [career_data[code] for code in personality_data.values()]
})

# Encode career labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['preferred_career'])

# Vectorize personality descriptions
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['personality_description'])

# Train model
model = MultinomialNB()
model.fit(X, y)

# ===== User interaction =====
print("🔍 Welcome to the Career Recommender!")
print("\n👇 Choose the description that matches you the most:\n")

# Display options
for i, description in enumerate(df["personality_description"], start=1):
    print(f"{i}. {description}")

# Get user input
while True:
    try:
        choice = int(input("\nEnter the number that best describes you (1–14): "))
        if 1 <= choice <= len(df):
            break
        else:
            print("⚠️ Please enter a valid number from the list.")
    except:
        print("⚠️ Please enter a number only.")

# Map choice to personality and predict
selected_description = df.iloc[choice - 1]["personality_description"]
input_vector = vectorizer.transform([selected_description])
prediction = model.predict(input_vector)
career = label_encoder.inverse_transform(prediction)[0]

# 🎯 Show result
print(f"\n🎉 Based on your personality, a great career choice for you is: **{career}**")


🔍 Welcome to the Career Recommender!

👇 Choose the description that matches you the most:

1. Strategic and Logical (likes planning and analyzing)
2. Creative and Outgoing (enjoys new ideas and people)
3. Hands-on and Practical (good with tools and machines)
4. Caring and Social (loves helping others)
5. Imaginative and Quiet (writes or creates deeply)
6. Curious and Energetic (loves solving problems)
7. Helpful and Organized (likes structure and stability)
8. Efficient and Direct (natural leader)
9. Confident and Driven (leads with vision)
10. Empathetic and Insightful (understands people deeply)
11. Fun and Outgoing (loves performance and spotlight)
12. Gentle and Artistic (prefers calm and expression)
13. Bold and Active (enjoys action and business)
14. Detail-oriented and Responsible (follows rules and tasks)

🎉 Based on your personality, a great career choice for you is: **Psychologist**


In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Embedded dataset
data = {
    "personality_type": [
        "INTJ", "ENFP", "ISTP", "ESFJ", "INFP",
        "ENTP", "ISFJ", "ESTJ", "ENTJ", "INFJ",
        "ESFP", "ISFP", "ESTP", "ISTJ", "ENFJ", "INTP"
    ],
    "preferred_career": [
        "Software Engineer", "Marketing Manager", "Mechanical Engineer", "Teacher", "Writer",
        "Entrepreneur", "Nurse", "Project Manager", "Executive", "Psychologist",
        "Performer", "Artist", "Sales Executive", "Accountant", "HR Manager", "Data Scientist"
    ]
}

df = pd.DataFrame(data)

# Encode the output labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['preferred_career'])

# Vectorize the MBTI types
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['personality_type'])

# Train model
model = MultinomialNB()
model.fit(X, y)

# ==== Mini MBTI-style quiz to estimate personality type ====
def ask_question(prompt, option1, option2):
    while True:
        ans = input(f"{prompt}\n(a) {option1}\n(b) {option2}\nYour choice (a/b): ").strip().lower()
        if ans == 'a':
            return '1'
        elif ans == 'b':
            return '2'
        else:
            print("❗ Please enter 'a' or 'b'.")

print("\n🧠 Answer these 4 quick questions to know your personality-based career:")

# E vs I
ei = ask_question(
    "Do you prefer being in a group over being alone?",
    "Yes, I love socializing (Extrovert)", 
    "No, I prefer quiet time (Introvert)"
)

# S vs N
sn = ask_question(
    "Do you rely more on facts or imagination?",
    "Facts and reality (Sensing)", 
    "Ideas and possibilities (Intuition)"
)

# T vs F
tf = ask_question(
    "When making decisions, do you go by logic or emotions?",
    "Logic and rules (Thinking)", 
    "Feelings and people (Feeling)"
)

# J vs P
jp = ask_question(
    "Do you like to have plans or go with the flow?",
    "Organized and scheduled (Judging)", 
    "Spontaneous and flexible (Perceiving)"
)

# Convert answers to MBTI type
mbti_map = {
    '1': ['E', 'S', 'T', 'J'],
    '2': ['I', 'N', 'F', 'P']
}

mbti_type = mbti_map[ei][0] + mbti_map[sn][1] + mbti_map[tf][2] + mbti_map[jp][3]
print(f"\n🔍 Your estimated MBTI personality type is: **{mbti_type}**")

# Predict career
try:
    input_vector = vectorizer.transform([mbti_type])
    prediction = model.predict(input_vector)
    career = label_encoder.inverse_transform(prediction)[0]
    print(f"\n🎯 Based on your personality, a great career choice could be: **{career}**")
except:
    print("Sorry, we couldn't match your personality type. Try again with different choices.")



🧠 Answer these 4 quick questions to know your personality-based career:
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.
❗ Please enter 'a' or 'b'.

🔍 Your estimated MBTI personality type is: **ESTJ**

🎯 Based on your personality, a great career choice could be: **Project Manager**
